In [1]:
import nltk
import re
import pandas as pd
import numpy as np

In [2]:
sents = ['Вася читает мою книгу', 'Напиши какое-нибудь письмо', 'Этот веселый мальчик идет', 'Он любит читать всякие книги']

In [3]:
rules = """
    S -> NP VP | NP V | VP
    NP -> ADJ N | ADJ NP
    VP -> V NP | V VP | V N
    ADJ -> 'мою' | 'какое-нибудь' | 'Этот' | 'веселый' | 'всякие'
    N -> 'книгу' | 'письмо' | 'мальчик' | 'книги'
    NP -> 'Вася' | 'Он'
    V -> 'читает' | 'Напиши' | 'идет' | 'любит' | 'читать'
""".split('\n')

In [4]:
grammar = nltk.CFG.fromstring('\n'.join(rules))

In [5]:
def print_parses(parser, sentence):
    for elem in parser.parse(sentence.split()):
        print(elem)

### Продемонстрируем алгоритм CYK parser-а

Матрица в CYK должна быть трёхмерной, но ради удобства сделаем её двумерной, схлопнув по оси Z.

In [6]:
n = 4
sent = sents[0].split()
df = pd.DataFrame(np.empty((n + 1, n)), columns=range(1, n + 1), dtype=str)
df = df.reindex(index=df.index[::-1])
df[:] = ''

#### На нулевую строку поместим слова нашего предложения

In [7]:
df.iloc[n, :] = sent
df.style.set_properties(**{'text-align': 'center'})

,1,2,3,4
4,,,,
3,,,,
2,,,,
1,,,,
0,Вася,читает,мою,книгу


#### Заполним первый ряд в соответствии с алгоритмом, т.е. пройдёмся с шагом 1 по первому ряду и запишем все возможные нетерминальные вершины, которые переходят в соответствующие терминальные

In [8]:
df.iloc[n - 1, :] = np.array(('NP', 'V', 'ADJ', 'N'))
df.style.set_properties(**{'text-align': 'center'})

,1,2,3,4
4,,,,
3,,,,
2,,,,
1,NP,V,ADJ,N
0,Вася,читает,мою,книгу


#### Заполним второй ряд в соответствии с алгоритмом, т.е. пройдёмся с шагом 2 по второму ряду и найдём все возможные разбиения на 2 составляющих, поскольку мы исходим из предположения о том, что грамматика в нормальной форме Хомского. В данном случае нашли только одно разложение: NP -> Adj N

In [9]:
df.iloc[n - 2, :] = np.array(('', '', 'NP', ''))
df.style.set_properties(**{'text-align': 'center'})

,1,2,3,4
4,,,,
3,,,,
2,,,NP,
1,NP,V,ADJ,N
0,Вася,читает,мою,книгу


#### Заполним третий ряд в соответствии с алгоритмом, т.е. пройдёмся с шагом 3 по третьему ряду и найдём все возможные разбиения на 2 составляющих, поскольку мы исходим из предположения о то, что грамматика в нормальной форме Хомского. В данном случае нашли только одно разложение VP -> V NP

In [10]:
df.iloc[n - 3, :] = np.array(('', 'VP', '', ''))
df.style.set_properties(**{'text-align': 'center'})

,1,2,3,4
4,,,,
3,,VP,,
2,,,NP,
1,NP,V,ADJ,N
0,Вася,читает,мою,книгу


#### Наконец, проверим, что S правильно раскладывается и на последнем ряду. И действительно, S -> NP VP

In [11]:
df.iloc[n - 4, :] = np.array(('S', '', '', ''))
df.style.set_properties(**{'text-align': 'center'})

,1,2,3,4
4,S,,,
3,,VP,,
2,,,NP,
1,NP,V,ADJ,N
0,Вася,читает,мою,книгу


#### Поскольку в левой верхней ячейке S (True), то это предложение действительно порождается нашей грамматикой

### Теперь продемонстрируем алгоритм Earley	parser-а

#### При добавлении items не учитываем дубликаты, чтобы избежать бесконечного цикла

#### S(0): • Вася читает мою книгу

In [12]:
n = 4
m = 20
df = pd.DataFrame(np.empty((m, n)), columns=['(state no.)', 'Production', '(Origin)', 'Comment'], dtype=str)
df[:] = ''
i = 0
df.iloc[i, :] = np.array((i + 1, 'S -> • NP VP', '0', 'start rule'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'S -> • NP V', '0', 'start rule'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'S -> • VP', '0', 'start rule'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'NP -> • ADJ N', '0', 'predict from (1)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'NP -> • ADJ NP', '0', 'predict from (1)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "NP -> • 'Вася'", '0', 'predict from (1)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "NP -> • 'Он'", '0', 'predict from (1)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> • V NP", '0', 'predict from (1)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> • V VP", '0', 'predict from (1)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> • V N", '0', 'predict from (1)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'читает'", '0', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'Напиши'", '0', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'идет'", '0', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'любит'", '0', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'читать'", '0', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'мою'", '0', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'какое-нибудь'", '0', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'Этот'", '0', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'веселый'", '0', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'всякие'", '0', 'predict from (4)'))
"""
# Поскольку в ашей грамматике N никогда не может идти слева, опустим следующие items S(0)
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'книгу'", '0', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'письмо'", '0', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'мальчик'", '0', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'книги'", '0', 'predict from (4)'))
"""

df.style.set_properties(**{'text-align': 'center'})

,(state no.),Production,(Origin),Comment
0,1,S -> • NP VP,0,start rule
1,2,S -> • NP V,0,start rule
2,3,S -> • VP,0,start rule
3,4,NP -> • ADJ N,0,predict from (1)
4,5,NP -> • ADJ NP,0,predict from (1)
5,6,NP -> • 'Вася',0,predict from (1)
6,7,NP -> • 'Он',0,predict from (1)
7,8,VP -> • V NP,0,predict from (1)
8,9,VP -> • V VP,0,predict from (1)
9,10,VP -> • V N,0,predict from (1)


#### S(1): Вася • читает мою книгу

In [13]:
n = 4
m = 11
df = pd.DataFrame(np.empty((m, n)), columns=['(state no.)', 'Production', '(Origin)', 'Comment'], dtype=str)
df[:] = ''
i = 0
df.iloc[i, :] = np.array((i + 1, "NP -> 'Вася' •", '0', 'scan from S(0)(6)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'S -> NP • VP', '0', 'complete from (1) and S(0)(1)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'S -> NP • V', '0', 'complete from (1) and S(0)(2)'))
"""
Больше нет items из S(0), в которых NP было бы слева
"""
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> • V NP", '1', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> • V VP", '1', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> • V N", '1', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'читает'", '1', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'Напиши'", '1', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'идет'", '1', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'любит'", '1', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'читать'", '1', 'predict from (3)'))

df.style.set_properties(**{'text-align': 'center'})

,(state no.),Production,(Origin),Comment
0,1,NP -> 'Вася' •,0,scan from S(0)(6)
1,2,S -> NP • VP,0,complete from (1) and S(0)(1)
2,3,S -> NP • V,0,complete from (1) and S(0)(2)
3,4,VP -> • V NP,1,predict from (2)
4,5,VP -> • V VP,1,predict from (2)
5,6,VP -> • V N,1,predict from (2)
6,7,V -> • 'читает',1,predict from (3)
7,8,V -> • 'Напиши',1,predict from (3)
8,9,V -> • 'идет',1,predict from (3)
9,10,V -> • 'любит',1,predict from (3)


#### S(2) = Вася читает • мою книгу

In [14]:
n = 4
m = 26
df = pd.DataFrame(np.empty((m, n)), columns=['(state no.)', 'Production', '(Origin)', 'Comment'], dtype=str)
df[:] = ''
i = 0
df.iloc[i, :] = np.array((i + 1, "V -> 'читает' •", '1', 'scan from S(1)(7)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "S -> NP V •", '0', 'complete from (1) and S(0)(2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> V • NP", '1', 'complete from (1) and S(1)(4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> V • VP", '1', 'complete from (1) and S(1)(5)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> V • N", '1', 'complete from (1) and S(1)(6)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'NP -> • ADJ N', '2', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'NP -> • ADJ NP', '2', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "NP -> • 'Вася'", '2', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "NP -> • 'Он'", '2', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> • V NP", '2', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> • V VP", '2', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> • V N", '2', 'predict from (4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'книгу'", '2', 'predict from (5)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'письмо'", '2', 'predict from (5)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'мальчик'", '2', 'predict from (5)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'книги'", '2', 'predict from (5)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'мою'", '2', 'predict from (6)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'какое-нибудь'", '2', 'predict from (6)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'Этот'", '2', 'predict from (6)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'веселый'", '2', 'predict from (6)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'всякие'", '2', 'predict from (6)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'читает'", '2', 'predict from (10)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'Напиши'", '2', 'predict from (10)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'идет'", '2', 'predict from (10)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'любит'", '2', 'predict from (10)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "V -> • 'читать'", '2', 'predict from (10)'))

df.style.set_properties(**{'text-align': 'center'})

,(state no.),Production,(Origin),Comment
0,1,V -> 'читает' •,1,scan from S(1)(7)
1,2,S -> NP V •,0,complete from (1) and S(0)(2)
2,3,VP -> V • NP,1,complete from (1) and S(1)(4)
3,4,VP -> V • VP,1,complete from (1) and S(1)(5)
4,5,VP -> V • N,1,complete from (1) and S(1)(6)
5,6,NP -> • ADJ N,2,predict from (3)
6,7,NP -> • ADJ NP,2,predict from (3)
7,8,NP -> • 'Вася',2,predict from (3)
8,9,NP -> • 'Он',2,predict from (3)
9,10,VP -> • V NP,2,predict from (4)


#### S(3) = Вася читает мою • книгу

In [15]:
n = 4
m = 16
df = pd.DataFrame(np.empty((m, n)), columns=['(state no.)', 'Production', '(Origin)', 'Comment'], dtype=str)
df[:] = ''
i = 0
df.iloc[i, :] = np.array((i + 1, "ADJ -> 'мою' •", '2', 'scan from S(2)(17)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'NP -> ADJ • N', '2', 'complete from (1) and S(2)(6)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'NP -> ADJ • NP', '2', 'complete from (1) and S(2)(7)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'книгу'", '3', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'письмо'", '3', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'мальчик'", '3', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "N -> • 'книги'", '3', 'predict from (2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'NP -> • ADJ N', '3', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'NP -> • ADJ NP', '3', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "NP -> • 'Вася'", '3', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "NP -> • 'Он'", '3', 'predict from (3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'мою'", '3', 'predict from (8)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'какое-нибудь'", '3', 'predict from (8)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'Этот'", '3', 'predict from (8)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'веселый'", '3', 'predict from (8)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "ADJ -> • 'всякие'", '3', 'predict from (8)'))

df.style.set_properties(**{'text-align': 'center'})

,(state no.),Production,(Origin),Comment
0,1,ADJ -> 'мою' •,2,scan from S(2)(17)
1,2,NP -> ADJ • N,2,complete from (1) and S(2)(6)
2,3,NP -> ADJ • NP,2,complete from (1) and S(2)(7)
3,4,N -> • 'книгу',3,predict from (2)
4,5,N -> • 'письмо',3,predict from (2)
5,6,N -> • 'мальчик',3,predict from (2)
6,7,N -> • 'книги',3,predict from (2)
7,8,NP -> • ADJ N,3,predict from (3)
8,9,NP -> • ADJ NP,3,predict from (3)
9,10,NP -> • 'Вася',3,predict from (3)


#### S(4) = Вася читает мою книгу •

In [16]:
n = 4
m = 4
df = pd.DataFrame(np.empty((m, n)), columns=['(state no.)', 'Production', '(Origin)', 'Comment'], dtype=str)
df[:] = ''
i = 0
df.iloc[i, :] = np.array((i + 1, "N -> 'книгу' •", '3', 'scan from S(3)(4)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'NP -> ADJ N •', '2', 'complete from (1) and S(3)(2)'))
i += 1
df.iloc[i, :] = np.array((i + 1, "VP -> V NP •", '1', 'complete from (2) and S(2)(3)'))
i += 1
df.iloc[i, :] = np.array((i + 1, 'S -> NP VP •', '0', 'complete from (3) and S(1)(2)'))

df.style.set_properties(**{'text-align': 'center'})

,(state no.),Production,(Origin),Comment
0,1,N -> 'книгу' •,3,scan from S(3)(4)
1,2,NP -> ADJ N •,2,complete from (1) and S(3)(2)
2,3,VP -> V NP •,1,complete from (2) and S(2)(3)
3,4,S -> NP VP •,0,complete from (3) and S(1)(2)


#### Поскольку мы получили одно из самых верхних правил (S -> NP VP), это предложение порождается нашей грамматикой
#### Можно также построить схему Earley parser с помощью стандартных средств, но она будет укороченной:

In [17]:
cp_verbose = nltk.EarleyChartParser(grammar, trace=21, trace_chart_width=100)
print_parses(cp_verbose, "Вася читает мою книгу")

|.        Вася       .       читает      .        мою        .       книгу       .|
Leaf Init Rule:
|[-------------------]                   .                   .                   .| [0:1] 'Вася'
|.                   [-------------------]                   .                   .| [1:2] 'читает'
|.                   .                   [-------------------]                   .| [2:3] 'мою'
|.                   .                   .                   [-------------------]| [3:4] 'книгу'
Top Down Init Rule:
|>                   .                   .                   .                   .| [0:0] S  -> * NP VP
|>                   .                   .                   .                   .| [0:0] S  -> * NP V
|>                   .                   .                   .                   .| [0:0] S  -> * VP

* Processing queue: 0 

Predictor Rule:
|>                   .                   .                   .                   .| [0:0] VP -> * V NP
|>                   .                

### Убедимся в том, что наша грамматика работает. Воспользуемся EarleyChartParser-ом

In [18]:
cp = nltk.EarleyChartParser(grammar)
for sent in sents:
    print(sent + ':')
    print_parses(cp, sent)
    print()

Вася читает мою книгу:
(S (NP Вася) (VP (V читает) (NP (ADJ мою) (N книгу))))

Напиши какое-нибудь письмо:
(S (VP (V Напиши) (NP (ADJ какое-нибудь) (N письмо))))

Этот веселый мальчик идет:
(S (NP (ADJ Этот) (NP (ADJ веселый) (N мальчик))) (V идет))

Он любит читать всякие книги:
(S
  (NP Он)
  (VP (V любит) (VP (V читать) (NP (ADJ всякие) (N книги)))))



#### Результаты работы парсера совпадают с приведённым выше пошаговым разбором алгоритма

#### 3. Можно улучшить работу парсера с новыми словами, если использовать морфологический анализатор, который поможет определить часть речи слова и, тем самым, позволит добавлять правила перевода нетерминалов в терминальные новые слова, при это ключевые правила останутся неизменными, т.к. они не затрагивают терминальные веришны. К примеру, при нахождении нового прилагательного X можно будет просто добавить новое правило ADJ -> X. В результате предложение можно будет разобрать.

#### Также вместо того, чтобы для каждой формы слова (а уж тем более для слов с заглавной и со строчной буквы) добавлять в грамматику своё правило перехода нетерминальной вершины в терминальную форму слова, достаточно завести только одно правило перевода нетерминальной вершины в стандартную форму слова (для глаголов -- в инфинитив, для существительных -- в им. падеж ед. числа и т.д.)

#### Хотя на самом деле в правилах без исключений можно завести id частей речи, чтобы во время распар(с/ш)ивания предложения просто обращаться к морфологической информации (части речи) и в зависимости от этого разбирать предложение. К примеру, последние 4 правила в rules можно было бы переформулировать следующим образом:
##### ADJ -> id_adjective OR id_possessive_pronoun
##### N -> id_noun
##### NP -> id_noun_собственное OR id_pronoun
##### V -> id_verb
##### Тогда, получив предложение с глаголом 'читает' и видя правило V -> id_verb, мы сразу можем вывести правило V -> 'читает'

#### 4. Неоднозначный выбор есть там, где есть омонимия. К примеру, если мы возьмём форму "мой", то она может быть формой глагола или формой прилагательного (притяжательного местоимения). И тогда предложение "Я вижу мой красивый дом" будет разбираться двояко. Убедимся в этом, расширив грамматику и подав это предложение на вход EarleyChartParser-у.

In [19]:
# первое правило в своей грамматике я не использовал, однако для демонстрации омонимии оно хорошо подходит
add_rules = """
    S -> N V
    ADJ -> 'мой' | 'красивый'
    N -> 'дом' | 'блины' | 'пила'
    V -> 'вижу' | 'мой' | 'мою' | 'пила'
    NP -> 'Я'
""".split('\n')

In [20]:
new_grammar = nltk.CFG.fromstring('\n'.join(rules + add_rules))

In [21]:
new_cp = nltk.EarleyChartParser(new_grammar)
sent = 'Я вижу мой красивый дом'
print_parses(new_cp, sent)

(S (NP Я) (VP (V вижу) (VP (V мой) (NP (ADJ красивый) (N дом)))))
(S (NP Я) (VP (V вижу) (NP (ADJ мой) (NP (ADJ красивый) (N дом)))))


In [22]:
# Наша грамматика не порождает неконфигурациональные ИГ, однако порождает два глагола подряд (случай типа "люблю делать")
sent = "мою мою книгу"
print_parses(new_cp, sent)

(S (VP (V мою) (NP (ADJ мою) (N книгу))))
(S (VP (V мою) (VP (V мою) (N книгу))))


#### Омонимия глагола с существительным наблюдается в форме "печь". У предложения "Он любит печь (,) блины" будет 2 разбора

In [23]:
sent = "пила пила"
print_parses(new_cp, sent)

(S (N пила) (V пила))
(S (VP (V пила) (N пила)))
